In [13]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import haversine as hs
from datetime import datetime
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

## Load

In [1]:
import os
import pandas as pd
import requests
from datetime import datetime

os.chdir("/home/tema/10X/Week8/Logistic optimization/data")
os.getcwd()

'/home/tema/10X/Week8/Logistic optimization/data'

In [2]:
drivers_location = pd.read_csv('drivers_location.csv')
completed_orders = pd.read_csv('completed_orders.csv')

In [3]:
drivers_location.head()

,id,order_id,driver_id,driver_action,lat,lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800


In [8]:
completed_orders.dtypes

Trip ID                    int64
Trip Origin               object
Trip Destination          object
Trip Start Time           object
Trip End Time             object
Trip Duration            float64
Origin Latitude          float64
Origin Longitude         float64
Destination Latitude     float64
Destination Longitude    float64
dtype: object

In [10]:
completed_orders.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,6.508813,3.377403,6.650970,3.345031
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,6.431671,3.455538,6.428081,3.472189
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,6.631679,3.338898,6.508324,3.359040
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,6.572757,3.367708,6.584881,3.361407
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615


In [4]:
merged_df= pd.read_csv('merged.csv')
merged_df.head()

,id,order_id,driver_id,driver_action,lat,lng,Trip ID,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,1,392001,243828,accepted,6.602207,3.270465,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615
1,2,392001,243588,rejected,6.592097,3.287445,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615
2,3,392001,243830,rejected,6.596133,3.281784,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615
3,4,392001,243539,rejected,6.596142,3.280526,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615
4,5,392001,171653,rejected,6.609232,3.288800,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615


In [9]:
merged_df.dtypes

id                         int64
order_id                   int64
driver_id                  int64
driver_action             object
lat                      float64
lng                      float64
Trip ID                    int64
Trip Start Time           object
Trip End Time             object
Trip Duration            float64
Origin Latitude          float64
Origin Longitude         float64
Destination Latitude     float64
Destination Longitude    float64
dtype: object

In [11]:
merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time'])
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time'])

completed_orders['Trip Start Time'] = pd.to_datetime(completed_orders['Trip Start Time'])
completed_orders['Trip End Time'] = pd.to_datetime(completed_orders['Trip End Time'])

In [13]:
completed_orders.dtypes, merged_df.dtypes

(Trip ID                           int64
 Trip Origin                      object
 Trip Destination                 object
 Trip Start Time          datetime64[ns]
 Trip End Time            datetime64[ns]
 Trip Duration                   float64
 Origin Latitude                 float64
 Origin Longitude                float64
 Destination Latitude            float64
 Destination Longitude           float64
 dtype: object,
 id                                int64
 order_id                          int64
 driver_id                         int64
 driver_action                    object
 lat                             float64
 lng                             float64
 Trip ID                           int64
 Trip Start Time          datetime64[ns]
 Trip End Time            datetime64[ns]
 Trip Duration                   float64
 Origin Latitude                 float64
 Origin Longitude                float64
 Destination Latitude            float64
 Destination Longitude           float64


In [14]:
merged_df.rename(columns={'Trip Duration': 'Trip Duration(minutes)'}, inplace=True)


In [15]:
first_date = completed_orders['Trip Start Time'].min()
last_date = completed_orders['Trip End Time'].max()

print("First Date:", first_date)
print("Last Date:", last_date)


First Date: 2021-07-01 06:21:02
Last Date: 2022-06-13 10:42:15


In [16]:
merged_first_date = merged_df['Trip Start Time'].min()
merged_last_date = merged_df['Trip End Time'].max()

print("First Date:", merged_first_date)
print("Last Date:", merged_last_date)

First Date: 2021-07-01 06:39:51
Last Date: 2021-08-04 16:50:57


## Compute Distances and Driving Speed
We will use the geopy library to compute the geodesic distance and the haversine function to compute the haversine distance. We will also calculate the speed based on these distances and timestamps.

In [5]:

# Helper function to compute geodesic distance
def compute_geodesic_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters


In [6]:
# Helper function to compute haversine distance
def compute_haversine_distance(coord1, coord2):
    return hs.haversine(coord1, coord2) * 1000  # Convert to meters


In [7]:
# Helper function to compute speed
def compute_speed(distance, start_time, end_time):
    time_diff = (end_time - start_time).total_seconds() / 3600.0  # in hours
    if time_diff == 0:
        return 0
    return distance / time_diff

In [9]:
merged_df['Geodesic Distance (m)'] = merged_df.apply(
    lambda row: compute_geodesic_distance((row['Origin Latitude'], row['Origin Longitude']),
                                          (row['Destination Latitude'], row['Destination Longitude'])), axis=1)


In [10]:
merged_df['Haversine Distance (m)'] = merged_df.apply(
    lambda row: compute_haversine_distance((row['Origin Latitude'], row['Origin Longitude']),
                                          (row['Destination Latitude'], row['Destination Longitude'])), axis=1)




In [ ]:
merged_df['Speed (m/s)'] = merged_df.apply(
    lambda row: compute_speed(row['Geodesic Distance (m)'], row['Trip Start Time'], row['Trip End Time']), axis=1)

merged_df.head()

## Compute Number of Riders and Orders within 500m Circles

In [ ]:
# Helper function to count riders/orders within a 500m radius
def count_within_radius(df, origin, radius=500):
    origin_coords = (origin['lat'], origin['lng'])
    return df.apply(lambda row: compute_haversine_distance(origin_coords, (row['lat'], row['lng'])) <= radius, axis=1).sum()



In [ ]:
accepted_orders = merged_df[merged_df['driver_action'] == 'accepted']
unfulfilled_orders = merged_df[merged_df['driver_action'] != 'accepted']

In [ ]:
merged_df['Accepted Riders within 500m'] = merged_df.apply(
    lambda row: count_within_radius(accepted_orders, row), axis=1)


In [ ]:
merged_df['Unfulfilled Orders within 500m'] = merged_df.apply(
    lambda row: count_within_radius(unfulfilled_orders, row), axis=1)


In [ ]:
print(merged_df[['order_id', 'Accepted Riders within 500m', 'Unfulfilled Orders within 500m']].head())

## Compute Clusters of Delivery Starting Locations and Destinations
We will use the KMeans clustering algorithm from scikit-learn to find clusters.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Extracting starting and destination coordinates
starting_coords = merged_df[['Origin Latitude', 'Origin Longitude']].drop_duplicates().to_numpy()
destination_coords = merged_df[['Destination Latitude', 'Destination Longitude']].drop_duplicates().to_numpy()

# Clustering starting locations
kmeans_start = KMeans(n_clusters=10, random_state=0).fit(starting_coords)
merged_df['Start Cluster'] = kmeans_start.predict(merged_df[['Origin Latitude', 'Origin Longitude']])

# Clustering destination locations
kmeans_dest = KMeans(n_clusters=10, random_state=0).fit(destination_coords)
merged_df['Destination Cluster'] = kmeans_dest.predict(merged_df[['Destination Latitude', 'Destination Longitude']])

print(merged_df[['order_id', 'Start Cluster', 'Destination Cluster']].head())


# Feature Engineering

Create new features based on time and location, such as trip duration, day of the week, hour of the day, and whether the trip occurred during a holiday or special event. Compute distances, driving speed, and other relevant metrics.



In [20]:
merged_df.columns

Index(['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng', 'Trip ID',
       'Trip Start Time', 'Trip End Time', 'Trip Duration', 'Origin Latitude',
       'Origin Longitude', 'Destination Latitude', 'Destination Longitude'],
      dtype='object')

In [25]:
merged_df['order_id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 771336 entries, 0 to 771335
Series name: order_id
Non-Null Count   Dtype
--------------   -----
771336 non-null  int64
dtypes: int64(1)
memory usage: 5.9 MB


In [ ]:
# Feature Engineering
# Create new features: Day of the week, Hour of the day, and Trip Duration in minutes
merged_df['Day of Week'] = merge_df['Trip Start Time'].dt.dayofweek
merged_df['Hour of Day'] = merged_df['Trip Start Time'].dt.hour

# Display the first few rows to confirm the new features
new_features_sample = merged_df[['Trip ID', 'Trip Start Time', 'Trip End Time', 'Trip Duration', 'Day of Week', 'Hour of Day']].head()

new_features_sample

In [26]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

In [34]:
# Count the number of rejections per driver
rejections = merged_df[merged_df['driver_action'] == 'rejected']
rejections_per_driver = rejections['driver_id'].value_counts()

# Display the results
rejections_per_driver.head(), rejections_per_driver.tail()

(driver_id
 171165    4514
 243768    4036
 245538    3643
 243417    3459
 244007    3453
 Name: count, dtype: int64,
 driver_id
 243502    1
 243620    1
 245688    1
 244225    1
 242876    1
 Name: count, dtype: int64)

In [37]:
accepted = merged_df[merged_df['driver_action'] == 'accepted']
#accepted

In [27]:
import numpy as np

# Calculate the distance between origins and destinations
merged_df['Trip Distance (km)'] = haversine_distance(
    merged_df['Origin Latitude'], merged_df['Origin Longitude'],
    merged_df['Destination Latitude'], merged_df['Destination Longitude']
)



In [31]:
# Display the first few rows to confirm the new feature
distance_df = merged_df[['Trip ID',
                         'driver_id',
                         'Origin Latitude',
                         'Origin Longitude',
                         'Destination Latitude',
                         'Destination Longitude',
                         'Trip Distance (km)']]

distance_df.head()

,Trip ID,driver_id,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,Trip Distance (km)
0,392001,243828,6.601042,3.276634,6.450107,3.391615,21.048262
1,392001,243588,6.601042,3.276634,6.450107,3.391615,21.048262
2,392001,243830,6.601042,3.276634,6.450107,3.391615,21.048262
3,392001,243539,6.601042,3.276634,6.450107,3.391615,21.048262
4,392001,171653,6.601042,3.276634,6.450107,3.391615,21.048262


In [38]:
merged_df.dtypes

id                                int64
order_id                          int64
driver_id                         int64
driver_action                    object
lat                             float64
lng                             float64
Trip ID                           int64
Trip Start Time          datetime64[ns]
Trip End Time            datetime64[ns]
Trip Duration                   float64
Origin Latitude                 float64
Origin Longitude                float64
Destination Latitude            float64
Destination Longitude           float64
Trip Distance (km)              float64
dtype: object

In [39]:
# Import module
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")
 
# Assign Latitude & Longitude
Latitude = merged_df['Origin Latitude'][0]
Longitude = merged_df['Origin Longitude'][0]
 


In [40]:
# Displaying Latitude and Longitude
print("Latitude: ", Latitude)
print("Longitude: ", Longitude)
 

Latitude:  6.6010417
Longitude:  3.2766339


In [41]:

# Get location with geocode
location = geolocator.reverse(f"{Latitude},{Longitude}")
 
# Display location
print("\nLocation of the given Latitude and Longitude:")
print(location)


Location of the given Latitude and Longitude:
Fadayomi Street, 100265, Lagos, Nigeria


## Holidays in Nigeria

In [45]:
# Merge the weather data with trip data on the date
merged_df['Trip Date'] = merged_df['Trip Start Time'].dt.strftime('%Y-%m-%d %H:00:00')
merged_df['Trip Date']


0         2021-07-01 09:00:00
1         2021-07-01 09:00:00
2         2021-07-01 09:00:00
3         2021-07-01 09:00:00
4         2021-07-01 09:00:00
                 ...         
771331    2021-07-06 10:00:00
771332    2021-07-06 10:00:00
771333    2021-07-06 10:00:00
771334    2021-07-06 10:00:00
771335    2021-07-06 10:00:00
Name: Trip Date, Length: 771336, dtype: object

In [46]:
import holidays

nigerian_holidays = holidays.country_holidays(country='NG', years=[2021, 2022])

nigerian_holidays.items()



dict_items([(datetime.date(2021, 1, 1), "New Year's Day"), (datetime.date(2021, 4, 2), 'Good Friday'), (datetime.date(2021, 4, 5), 'Easter Monday'), (datetime.date(2021, 5, 1), "Workers' Day"), (datetime.date(2021, 6, 12), 'Democracy Day'), (datetime.date(2021, 10, 1), 'Independence Day'), (datetime.date(2021, 12, 25), 'Christmas Day'), (datetime.date(2021, 12, 26), 'Boxing Day'), (datetime.date(2021, 5, 13), 'Eid-el-Fitr (estimated)'), (datetime.date(2021, 5, 14), 'Eid-el-Fitr Holiday (estimated)'), (datetime.date(2021, 7, 20), 'Eid-el-Kabir (estimated)'), (datetime.date(2021, 7, 21), 'Eid-el-Kabir Holiday (estimated)'), (datetime.date(2021, 10, 18), 'Eid-el-Mawlid (estimated)'), (datetime.date(2021, 5, 3), "Workers' Day (observed)"), (datetime.date(2021, 6, 14), 'Democracy Day (observed)'), (datetime.date(2021, 12, 27), 'Christmas Day (observed)'), (datetime.date(2021, 12, 28), 'Boxing Day (observed)'), (datetime.date(2022, 1, 1), "New Year's Day"), (datetime.date(2022, 4, 15), 'Good

In [48]:
for date, holidays in nigerian_holidays.items():
    print(type(date), date, holidays)

<class 'datetime.date'> 2021-01-01 New Year's Day
<class 'datetime.date'> 2021-04-02 Good Friday
<class 'datetime.date'> 2021-04-05 Easter Monday
<class 'datetime.date'> 2021-05-01 Workers' Day
<class 'datetime.date'> 2021-06-12 Democracy Day
<class 'datetime.date'> 2021-10-01 Independence Day
<class 'datetime.date'> 2021-12-25 Christmas Day
<class 'datetime.date'> 2021-12-26 Boxing Day
<class 'datetime.date'> 2021-05-13 Eid-el-Fitr (estimated)
<class 'datetime.date'> 2021-05-14 Eid-el-Fitr Holiday (estimated)
<class 'datetime.date'> 2021-07-20 Eid-el-Kabir (estimated)
<class 'datetime.date'> 2021-07-21 Eid-el-Kabir Holiday (estimated)
<class 'datetime.date'> 2021-10-18 Eid-el-Mawlid (estimated)
<class 'datetime.date'> 2021-05-03 Workers' Day (observed)
<class 'datetime.date'> 2021-06-14 Democracy Day (observed)
<class 'datetime.date'> 2021-12-27 Christmas Day (observed)
<class 'datetime.date'> 2021-12-28 Boxing Day (observed)
<class 'datetime.date'> 2022-01-01 New Year's Day
<class 'd

In [49]:
len(nigerian_holidays.items())

38

In [50]:
# nigerian_holidays.get('2021-4-2')
"2022, 10, 01" in nigerian_holidays


True

In [52]:
merged_df.head(2)

,id,order_id,driver_id,driver_action,lat,lng,Trip ID,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,Trip Distance (km),Trip Date
0,1,392001,243828,accepted,6.602207,3.270465,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,21.048262,2021-07-01 09:00:00
1,2,392001,243588,rejected,6.592097,3.287445,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,21.048262,2021-07-01 09:00:00


In [53]:
merged_df['Is Holiday'] = merged_df['Trip Date'].apply(lambda x: int(x in nigerian_holidays))

In [62]:
merged_df['Is Holiday'].unique()


array([0])

In [60]:
merged_df['Is Weekend'] = merged_df['Day of Week'].apply(lambda x: 1 if x in [5, 6] else 0)

KeyError: 'Day of Week'